In [1]:
#importing the necessary libraries
import firebase_admin
from firebase_admin import auth,firestore,storage,credentials

In [2]:
#initialising the credentials
cred = credentials.Certificate("tinder-api-307fa-firebase-adminsdk-12d8z-48329730a7.json")
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred)

In [3]:
store=firestore.client()

In [4]:
#creating new user
def signUp(email_id,passwd):
    uid=""
    message=""
    try:
        user = auth.create_user(
        email=email_id,
        email_verified=False,
        password=passwd,
        disabled=False)
        uid=user.uid
        message="Sucessfully created new user:"
    except:
        message='User Already Exists'
    return {'Message':message,"UserID":uid}

In [5]:
#SignUp 
email_id='ansar326629@gmail.com'
passwd='123456'

signUp(email_id,passwd)

{'Message': 'User Already Exists', 'UserID': ''}

In [6]:
# Login in with userid
def login(email_id,passwd):
    uid=""
    message=""
    try:
        user = auth.get_user_by_email(email_id)
        uid=user.uid
        message="Succesfully login !!!"
    except:
        message="Email does not exist !!!"
    return {"Message":message,"Userid":uid}

In [7]:
#Login
login(email_id,passwd)

{'Message': 'Succesfully login !!!', 'Userid': 'aM7Ag4lfZuU0T5IR1fVViCiqOXx2'}

In [8]:
#Updating user data
def update_user_data(uid,user_data):
    user_details={}
    user_details['name']=user_data['name']
    user_details['phone_number']=user_data['phone_number']
    user_details['image']=user_data['image']
    user_details['desp']=user_data['desp']
    user_details['location']=user_data['location']
    user_details['dob']=user_data['dob']
    user_details['gender']=user_data['gender']
    user_details['passion']=user_data['passion']
    user_details['job']=user_data['job']
    user_details['company']=user_data['company']
    user_details['Created At']=firestore.SERVER_TIMESTAMP
    store.collection("Users").document(uid).set(user_details)
    

In [9]:
#Getting User data 
user_data={}
user_data['name']="Ansari Mohd Ali"
user_data['phone_number']="09876543219"
user_data['image']="https://firebasestorage.googleapis.com/v0/b/tinder-api-307fa.appspot.com/o/photo_1.jpg?alt=media&token=0f9728b7-d4a3-4517-baa3-447edb9fbcac"
user_data['desp']="Looking for friends"
user_data['location']={
    'coordinate':{'lat':19.1868,'lat':72.9545},
    'country':"India",
    'state':"Maharashtra",
    'city':"Thane"
}
user_data['dob']="10/05/1997"
user_data['gender']="male"
user_data['passion']="Reading, Writing, Cycling"
user_data['job']="Student"
user_data['company']="Somewhere in Thane"

In [10]:
update_user_data("aM7Ag4lfZuU0T5IR1fVViCiqOXx2",user_data)

In [11]:
#for swipe
def swipe(userid_A,userid_B,is_right,is_left):
    result={}
    result['uid_a']=userid_A
    result['uid_b']=userid_B
    result['is_right']=is_right
    result['is_left']=is_left
    store.collection('Swipes').document(userid_A).set(result)

In [12]:
userid_A="aM7Ag4lfZuU0T5IR1fVViCiqOXx2"
userid_B="gT9W5jX0r9apcItvBeove9NaMxw2"
is_right=True
is_left=False
swipe(userid_A,userid_B,is_right,is_left)

In [13]:
userid_B="aM7Ag4lfZuU0T5IR1fVViCiqOXx2"
userid_A="gT9W5jX0r9apcItvBeove9NaMxw2"
is_right=True
is_left=False
swipe(userid_A,userid_B,is_right,is_left)

In [14]:
#getting match
def match(userid_A,userid_B,is_right,is_left):
    result={}
    docs=store.collection('Swipes').stream()
    for doc in docs:
        if (userid_A in doc.id or userid_B in doc.id) and (doc.to_dict().get('is_right')):
            result['uid_a']=userid_A
            result['uid_b']=userid_B
            result['Matched']=True
            store.collection('Matched').document('Yes').set(result)
            return 'Congratulations !!! you both matched'
        else:
            result['uid_a']=userid_A
            result['uid_b']=userid_B
            result['Matched']=False
            store.collection('Matched').document('No').set(result)
            return 
    

In [15]:
match(userid_A,userid_B,is_right,is_left)

'Congratulations !!! you both matched'

In [16]:
#getting feed based on city
def get_feed(uid,city,gender):
    docs=store.collection('Users').where("gender","==",gender).stream()
    swipes=store.collection('Swipes').stream()
    visited=[]
    for swipe in swipes:
        if (swipe.to_dict().get('uid_a')==uid):
            visited.append(swipe.to_dict().get('uid_b'))
    result={}
    #if person is not visited once
    for doc in docs:
        if (doc.id not in visited) and (doc.to_dict().get('location').get('city')==city):
            result[doc.id]=doc.to_dict()
    return result

In [20]:
result=get_feed('DjJgiLqiFSRpiMOLbPfRMwqYUvi1','Thane','male')
result

{'aM7Ag4lfZuU0T5IR1fVViCiqOXx2': {'Created At': DatetimeWithNanoseconds(2021, 3, 23, 13, 20, 54, 712000, tzinfo=<UTC>),
  'image': 'https://firebasestorage.googleapis.com/v0/b/tinder-api-307fa.appspot.com/o/photo_1.jpg?alt=media&token=0f9728b7-d4a3-4517-baa3-447edb9fbcac',
  'dob': '10/05/1997',
  'desp': 'Looking for friends',
  'passion': 'Reading, Writing, Cycling',
  'phone_number': '09876543219',
  'gender': 'male',
  'location': {'coordinate': {'lat': 72.9545},
   'state': 'Maharashtra',
   'country': 'India',
   'city': 'Thane'},
  'company': 'Somewhere in Thane',
  'job': 'Student',
  'name': 'Ansari Mohd Ali'}}

In [19]:
#1.46.23